# ベイズロジスティック回帰

ベイズ線形回帰の場合と同様に入力xと出力yの間の関係を仮定しますが、非線形な関係を仮定します。出力が真理値のような2値的な量の場合にその事後分布を推定することに相当します。

In [7]:
import edward as ed
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
%matplotlib inline

Couldn't import dot_parser, loading of dot files will not be possible.


# Data

以下のように入力$y=tanh(x)$ $(x \in R, y\in R)$となるような関係を仮定します。

In [8]:
N = 40  # number of data points
D = 1  # number of features

def build_toy_dataset(N, noise_std=0.1):
  D = 1
  X = np.linspace(-6, 6, num=N)
  y = np.tanh(X) + np.random.normal(0, noise_std, size=N)
  y[y < 0.5] = 0
  y[y >= 0.5] = 1
  X = (X - 4.0) / 4.0
  X = X.reshape((N, D))
  return X, y

訓練データ(X_train,y_train)と学習されたパラメータを検証するためのデータ(X_test,y_test)を生成します。

In [9]:
X_train, y_train = build_toy_dataset(N)

In [10]:
X_test, y_test = build_toy_dataset(N)

# Model

In [ ]:
X = tf.placeholder(tf.float32, [N, D])
w = Normal(loc=tf.zeros(D), scale=3.0 * tf.ones(D))
b = Normal(loc=tf.zeros([]), scale=3.0 * tf.ones([]))
y = Bernoulli(logits=ed.dot(X, w) + b)

入力変数Xをplaceholderで定義し、重みw,バイアスbがあるのは線形回帰と同様ですが出力yは確率的に0,1の2値を返すベルヌーイ(Bernoulli)分布でかかれます。

$y \sim Bern(w\cdot X+b)$

# 推定

In [ ]:
qw_loc = tf.Variable(tf.random_normal([D]))
qw_scale = tf.nn.softplus(tf.Variable(tf.random_normal([D])))
qb_loc = tf.Variable(tf.random_normal([]) + 10)
qb_scale = tf.nn.softplus(tf.Variable(tf.random_normal([])))

qw = Normal(loc=qw_loc, scale=qw_scale)
qb = Normal(loc=qb_loc, scale=qb_scale)